### tensorflow_실전예제

In [1]:
import tensorflow as tf
import numpy as np

# keras가 tensorflow의 core
from tensorflow.keras.datasets.cifar10 import load_data

In [2]:
# 파이선 장점: 데이터가 입력될 때, 타입이 결정됨
# 단점: 데이터를 대입하지 않으면, 프로그램 에러 종료(초기화되지 않은 포인터 호출가 동일 현상)
# 변수-> 주소값 -> 데이터

def next_batch(num, data, labels):
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    
    # for in문: 인덱스 에러를 방지
    # i 값은 실제 데이터
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    
    # 데이터 결과값은 리스트 형식 -> array 로 변환
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [16]:
def build_CNN_classifier(x):

  x_image = x

  # Variable: 가중치를 저장
  # 정규분포의 2배수 내의 값만 가능 -> 필터
  # 5x5가 64개, 3=컬러, rgb
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
  
  # 바이어스
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))

  # 입력사이즈와 같게 양쪽에 2개씩 패딩
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

  # padding size: 3, stride: 2
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

  # 가중치가 있으면, 꼭  bias고려
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')


  W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)


  W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv4 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

  W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
  b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
  h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)


  W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))
  
  # FFNN
  h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
  h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
  
  # 2nd Layer
  W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2)) #출력 특징 10 
  b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits


# 이미지 사이즈 32x32x3: RGB
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])

# one-hot-encoding
y = tf.placeholder(tf.float32, shape=[None, 10])

# dropout 값 저장
keep_prob = tf.placeholder(tf.float32)

# 훈련용, 테스트 데이터로 분할
(x_train, y_train), (x_test, y_test) = load_data() # keras image_generator 사용

# squeeze 축을 줄일 때 사용
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# 예측된 값, cost값
y_pred, logits = build_CNN_classifier(x)


# 비용함수
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

# RMSPropOptimizer 0.001
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)


correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10000): # epoch
    batch = next_batch(128, x_train, y_train_one_hot.eval())


    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
      loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

      print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.8})

  test_accuracy = 0.0 
  for i in range(10):
    test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
    test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0})
  test_accuracy = test_accuracy / 10;
  print("테스트 데이터 정확도: %f" % test_accuracy)
 

170500096/170498071 [==============================] - 1825s 11us/step


W0921 16:40:09.748905  7628 deprecation.py:506] From <ipython-input-16-b8f1489c19ac>:42: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0921 16:40:09.765860  7628 deprecation.py:323] From <ipython-input-16-b8f1489c19ac>:67: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0921 16:40:09.884542  7628 deprecation.py:506] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\python\training\rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
In

반복(Epoch): 0, 트레이닝 데이터 정확도: 0.164062, 손실 함수(loss): 164.321320
반복(Epoch): 100, 트레이닝 데이터 정확도: 0.273438, 손실 함수(loss): 2.109767
반복(Epoch): 200, 트레이닝 데이터 정확도: 0.289062, 손실 함수(loss): 1.939300
반복(Epoch): 300, 트레이닝 데이터 정확도: 0.492188, 손실 함수(loss): 1.589643
반복(Epoch): 400, 트레이닝 데이터 정확도: 0.429688, 손실 함수(loss): 1.557354
반복(Epoch): 500, 트레이닝 데이터 정확도: 0.460938, 손실 함수(loss): 1.469646
반복(Epoch): 600, 트레이닝 데이터 정확도: 0.429688, 손실 함수(loss): 1.427005
반복(Epoch): 700, 트레이닝 데이터 정확도: 0.476562, 손실 함수(loss): 1.407651
반복(Epoch): 800, 트레이닝 데이터 정확도: 0.601562, 손실 함수(loss): 1.186148
반복(Epoch): 900, 트레이닝 데이터 정확도: 0.562500, 손실 함수(loss): 1.244777
반복(Epoch): 1000, 트레이닝 데이터 정확도: 0.609375, 손실 함수(loss): 1.234111
반복(Epoch): 1100, 트레이닝 데이터 정확도: 0.617188, 손실 함수(loss): 1.219134
반복(Epoch): 1200, 트레이닝 데이터 정확도: 0.562500, 손실 함수(loss): 1.208992


KeyboardInterrupt: 